In [1]:
import numpy as np
import glob

import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('seaborn')

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from db.gateway.gateway import TracksGateway

In [2]:
tracks_gw = TracksGateway()
df = tracks_gw.fetch_all()
df.head()

,danceability,track_id,title,key,mode,acousticness,liveness,valence,duration_ms,artist,id,energy,loudness,speechiness,instrumentalness,tempo,time_signature
0,0.653,0aQA9DP54h37OevE7hRc2a,NaN,7.0,0.0,0.2400,0.1080,0.405,252712.0,NaN,1,0.796,-4.050,0.2450,0.00000,160.026,4.0
1,0.731,4ixBAI0E1M6df9mP27etZK,NaN,4.0,1.0,0.2010,0.1600,0.477,228056.0,NaN,2,0.494,-9.567,0.0467,0.00000,120.045,4.0
2,0.859,3jO5IAvslWxaTxswokg66i,NaN,3.0,1.0,0.2080,0.1140,0.370,195453.0,NaN,3,0.464,-7.328,0.2890,0.00000,134.966,4.0
3,0.482,5iS3TEmmRcDJU2pTJQ98zB,NaN,10.0,0.0,0.7750,0.1070,0.350,300861.0,NaN,4,0.617,-10.288,0.0869,0.58300,174.040,4.0
4,0.824,29xPIFa0othUcFrEGyJCZu,NaN,1.0,1.0,0.0236,0.0762,0.509,186973.0,NaN,5,0.684,-7.185,0.4070,0.00853,141.143,4.0


In [3]:
df.shape

(54775, 17)

In [4]:
# Max number of tracks take into account
n_total = len(glob.glob('./local/scraped_data/*/*'))*200
print(n_total)

0


**Wstępnie można zaobserwować występowanie tych samych utworów w ramach wielu rankingów.**

In [5]:
n_ranks = {}
country_codes = [path.split('/')[-1] for path in glob.glob('./local/scraped_data/*')]
for country_code in country_codes:
    n_ranks[country_code] = len(glob.glob(f'./local/scraped_data/{country_code}/*'))
    
n_ranks = sorted(n_ranks.items(), key=lambda x: x[1])
n_ranks = dict(n_ranks)

In [6]:
n_ranks

{}

**Informacje o rankingach są niepełne dla takich państw jak Białoruś (3), Indie (158), Emiraty Arabskie (172), Egipt (172) oraz Izrael (208).**

In [7]:
drop_cols = ['id', 'track_id', 'title', 'artist', 'key', 'mode', 'time_signature']
df_numeric = df.drop(drop_cols, axis=1)
features = list(df_numeric.columns)
x = df_numeric.to_numpy()
x = StandardScaler().fit_transform(x)

In [ ]:
sns.pairplot(df.sample(n=1000).drop(drop_cols, axis=1));

KeyboardInterrupt: 

**Cechy audio nie są skorelowane (zauważalna korelacja jedynie dla `energy` i `loudness`), charakteryzują się różnymi rozkładami.**

In [ ]:
pca = PCA()
x_pca = pca.fit_transform(x)

In [ ]:
plt.plot(pca.explained_variance_ratio_, 'bo')
plt.xlabel('Principal component number')
plt.ylabel('Explained variance')
plt.show()

**Poszczególne komponenty zachowują względnie dużo informacji - zbiór jest złożony.**

In [ ]:
pc1 = pca.components_[0]
pc2 = pca.components_[1]
print('Attribute, PC1, PC2')

projected_features = {}

for i in range(0, pc1.shape[0]):
    print(features[i] + ' : ' + repr(pc1[i]) + ' : ' + repr(pc2[i]))
    projected_features[features[i]] = (pc1[i], pc2[i])

**Wartości elementów są stosunkowo duże dla wszystkich cech, oprócz cechy `tempo`.**

In [ ]:
# Scale component values down for clarity
x_pca[:, 0] /= np.amax(x_pca[:, 0])
x_pca[:, 1] /= np.amax(x_pca[:, 1])

In [ ]:
plt.figure(figsize=(30, 20))
plt.scatter(x_pca[:, 0], x_pca[:, 1], c='k', s=1)

for feature_name, coords in projected_features.items():
    plt.annotate('', xy=coords, xytext=(0, 0), arrowprops=dict(arrowstyle='->', color='r', lw=2))
    plt.text(x=coords[0], y=coords[1], s=feature_name, fontsize=30, color='r')

plt.title('Projection by PCA')
plt.xlabel('1st component')
plt.ylabel('2nd component')
plt.ylim(-1.1, 1.1)
plt.xlim(-0.6, 1.1)
plt.show()

**Przestrzeń jest rozciągana głównie przez to, jak utwory są głośne i energiczne oraz jak bardzo są akustyczne.**